# Importing libraries

In [1]:
from time import time

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB


from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import re
import spacy


from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix



In [2]:
# Load Data
df_tools=pd.read_csv('TOOLS_COMPLETELY_CLEAN_FULLY_MERGED_RDY_FOR_MDLLING')

In [3]:
df_tools.head()

,title,brand,main_cat,price,product_id,Reviewer_id,rating,verified,reviewText,DATE
0,Breeding Organic Vegetables: A Step-by-Step Gu...,SioGreen,Tools & Home Improvement,45.902274,0982085028,AL19QO4XLBQPU,5.0,True,"returned, decided against this product",2018-01-28
1,Breeding Organic Vegetables: A Step-by-Step Gu...,SioGreen,Tools & Home Improvement,45.902274,0982085028,A1I7CVB7X3T81E,5.0,True,Awesome heater for the electrical requirements...,2017-11-30
2,Breeding Organic Vegetables: A Step-by-Step Gu...,SioGreen,Tools & Home Improvement,45.902274,0982085028,A1AQXO4P5U674E,5.0,True,Keeps the mist of your wood trim and on you. B...,2017-09-12
3,Breeding Organic Vegetables: A Step-by-Step Gu...,SioGreen,Tools & Home Improvement,45.902274,0982085028,AIRV678P7C4NK,4.0,True,"So far I hooked it up and tested it , filled a...",2017-07-19
4,Breeding Organic Vegetables: A Step-by-Step Gu...,SioGreen,Tools & Home Improvement,45.902274,0982085028,A22I5QDNTNECDW,1.0,True,"i installed this 10 months ago, instructions w...",2017-05-25


In [4]:
df_tools.shape

(2267345, 10)

In [5]:
df_tools.drop_duplicates(subset=None,keep='first',inplace=True)

In [6]:
df_tools.shape

(1817775, 10)

In [7]:
sentiment=[]
for i in df_tools['rating']:
    if (i>3):
        sentiment.append('Postive')
    elif(i<3):
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')

In [8]:
df_tools['ReviewSentiment']=sentiment

In [9]:
Text_Tools  = df_tools[["reviewText","ReviewSentiment"]]

In [10]:
Text_Tools.head()

,reviewText,ReviewSentiment
0,"returned, decided against this product",Postive
1,Awesome heater for the electrical requirements...,Postive
2,Keeps the mist of your wood trim and on you. B...,Postive
3,"So far I hooked it up and tested it , filled a...",Postive
4,"i installed this 10 months ago, instructions w...",Negative


In [11]:
Text_Tools.isnull().sum()

reviewText         0
ReviewSentiment    0
dtype: int64

In [12]:
Text_Tools.shape

(1817775, 2)

# TEXT PREPROCESSING 

In [13]:
import spacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
stopwords = en.Defaults.stop_words

print(len(stopwords))
#print(stopwords)

326


In [14]:
def clean_text(text):
    # Split the text into individual words
    words = text.split()
    
    # Remove any punctuation or non-alphabetic characters
    words = [word for word in words if word.isalpha()]
    
    # Convert all words to lowercase
    words = [word.lower() for word in words]
    
    # Remove any stop words
    stop_words = stopwords=('english')
    words = [word for word in words if word not in stop_words]
    
    # Join the words back into a single string
    return " ".join(words)


# Clean the "text" column using the clean_text() function
Text_Tools["reviewText"] = Text_Tools["reviewText"].apply(clean_text)

# Print the cleaned data
print(Text_Tools)

                                                reviewText ReviewSentiment
0                             decided against this product         Postive
1        awesome heater for the electrical makes an awe...         Postive
2         keeps the mist of your wood trim and on bendable         Postive
3        so far hooked it up and tested it filled a fiv...         Postive
4        installed this months instructions were easy a...        Negative
...                                                    ...             ...
2267340  the tool a handy drain especially for clearing...         Postive
2267341  this thing bathroom sink was so friend and dra...         Postive
2267342  purchased this exact product from a local box ...         Postive
2267343  have had one of these in the past and was curi...         Postive
2267344                    product works as thank you very         Postive

[1817775 rows x 2 columns]


In [15]:
Text_Tools["reviewText"]=Text_Tools["reviewText"].apply(lambda x:' '.join([word for word in x.split() if word not in stopwords]))

In [16]:
Text_Tools['reviewText']

0                                            decided product
1          awesome heater electrical makes awesome prehea...
2                              keeps mist wood trim bendable
3          far hooked tested filled gallon bucket hot per...
4          installed months instructions easy worked wate...
                                 ...                        
2267340    tool handy drain especially clearing hairballs...
2267341    thing bathroom sink friend drano cut maintenan...
2267342    purchased exact product local box bathroom sin...
2267343    past curious getting noted star minus inaccura...
2267344                                  product works thank
Name: reviewText, Length: 1817775, dtype: object

####  CHECKING OUR TEXT DATA FIRST 100 ROWS 

In [17]:
c = 0 
for i in Text_Tools["reviewText"]:
    print(i)
    c+=1
    if c==100:
        break

decided product
awesome heater electrical makes awesome preheater talnkless system
keeps mist wood trim bendable
far hooked tested filled gallon bucket hot perfect temp flow valve came broke tried tighten stop hooked directly sharkbite pex work fine hooked poll amp wait till rest bathroom finished better
installed months instructions easy worked water heater stuck tried different modes reset circuit heat water update warranty requires licensed useless service renders possible difficulty monetary gain company disturbs actually dropped heater fixed company named changed supposed heating element lasts years broken tried pick difficult pick possible worst try working voip callback system bad sound baby crying background gives away pretty product like kind repackaged alibaba celsius reading obviously usa jump hoops pick repaired left figured save problems road want deal bad service bought water installed require contractor install warranty new water heater readings dial adjusting inlet manu

In [18]:
Text_Tools['ReviewSentiment'] = '__label__' + Text_Tools['ReviewSentiment'].astype(str)
Text_Tools.head(5)

,reviewText,ReviewSentiment
0,decided product,__label__Postive
1,awesome heater electrical makes awesome prehea...,__label__Postive
2,keeps mist wood trim bendable,__label__Postive
3,far hooked tested filled gallon bucket hot per...,__label__Postive
4,installed months instructions easy worked wate...,__label__Negative


In [19]:
print(stopwords)

{'unless', 'our', 'less', 'empty', 'indeed', 'anyhow', 'noone', "'m", 'eleven', 'several', 'because', 'give', 'sixty', 'already', 'might', 'myself', 'yourself', 'herself', 'due', 'not', "'d", 'thence', 'three', 'nor', 'sometimes', '’s', 'beside', 'whereupon', 'around', 'both', '’d', 'put', 'call', '‘m', 'forty', 'hereupon', 'above', 'without', 'beforehand', 'nobody', 'perhaps', 'quite', 'ours', 'so', 'someone', 'amongst', 'only', 'she', 'such', 'he', 'even', 'enough', 'hers', 'one', '’m', 'ca', 'each', 'whereas', 'were', 'those', 'everyone', 'does', 'as', 'eight', 'been', 'much', 'show', 'there', 'with', 'many', 'at', 'you', 'nothing', 'therein', 'itself', 'its', 'wherein', 'has', 'least', 'whose', 'done', 'within', 'some', 'be', 'everything', 'seemed', 'hereby', 'did', 'never', 'per', 'very', 'after', 'yourselves', 'that', 'somehow', 'an', 'back', 'about', 'cannot', 'must', 'my', 'afterwards', 'n’t', 'by', 'of', 'move', '‘ll', 'wherever', 'however', 'which', 'fifteen', 'via', 'anythin

In [24]:
Text_Tools.head()

,reviewText,ReviewSentiment
0,decided product,__label__Postive
1,awesome heater electrical makes awesome prehea...,__label__Postive
2,keeps mist wood trim bendable,__label__Postive
3,far hooked tested filled gallon bucket hot per...,__label__Postive
4,installed months instructions easy worked wate...,__label__Negative


In [25]:
X=[]
y=[]
for i in Text_Tools['reviewText']:
    X.append(i)
for j in Text_Tools['ReviewSentiment']:
    y.append(j)  

In [26]:
print(type(X))

<class 'list'>


In [27]:
print(X[:10])

['decided product', 'awesome heater electrical makes awesome preheater talnkless system', 'keeps mist wood trim bendable', 'far hooked tested filled gallon bucket hot perfect temp flow valve came broke tried tighten stop hooked directly sharkbite pex work fine hooked poll amp wait till rest bathroom finished better', 'installed months instructions easy worked water heater stuck tried different modes reset circuit heat water update warranty requires licensed useless service renders possible difficulty monetary gain company disturbs actually dropped heater fixed company named changed supposed heating element lasts years broken tried pick difficult pick possible worst try working voip callback system bad sound baby crying background gives away pretty product like kind repackaged alibaba celsius reading obviously usa jump hoops pick repaired left figured save problems road want deal bad service bought water installed require contractor install warranty new water heater readings dial adjust

# TRAIN TEST SPLIT

In [28]:
# Splitting the data in 80:20 ratio for train and test respectively
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=9)

In [29]:
# print("X_train",X_train.shape)
# print("X_test",X_test.shape)
# print("y_train",y_train.shape)
# print("y_test",y_test.shape)

In [30]:
len(X_train)

1454220

# VECTORIZATION (TFIDF)

In [31]:
# Vectorizing the training data using TfidfVectorizer()
myvector=TfidfVectorizer()
train_vector=myvector.fit_transform(X_train)

In [32]:
train_vector.shape

(1454220, 109944)

In [33]:
# We see that a vocabulary of 110244 words are made

In [34]:
test_vector=myvector.transform(X_test)

In [35]:
test_vector.shape

(363555, 109944)

##### CLASSIFICATION USING NAIVE BAYES 

In [36]:
# Perform classification using MultinominalNB()
clf=MultinomialNB()
train=clf.fit(train_vector,y_train)

In [37]:
# Check for predicted value of y
y_pred=clf.predict(test_vector)

In [38]:
# Classification Report
print(classification_report(y_test, y_pred))

                   precision    recall  f1-score   support

__label__Negative       0.86      0.06      0.11     29993
 __label__Neutral       0.40      0.00      0.00     23977
 __label__Postive       0.86      1.00      0.92    309585

         accuracy                           0.86    363555
        macro avg       0.71      0.35      0.34    363555
     weighted avg       0.83      0.86      0.79    363555



## conclusion:- accuracy is 86%

In [39]:
# Confusion Matrix
confusion_matrix(y_test,y_pred)

array([[  1780,      3,  28210],
       [   155,     10,  23812],
       [   126,     12, 309447]], dtype=int64)

In [40]:
# Evaluation Metrics
print('accuracy:', accuracy_score(y_test, y_pred))
print('recall:', recall_score(y_test, y_pred, average='weighted'))
print('f1-score:', f1_score(y_test, y_pred, average='weighted'))
print('precision:', precision_score(y_test, y_pred,average='weighted'))

accuracy: 0.856093300876071
recall: 0.856093300876071
f1-score: 0.7945771922501816
precision: 0.8266275073755458


In [41]:
TF_IDF_ACC = round(accuracy_score(y_test, y_pred),2)*100

In [42]:
TF_IDF_PREC =  round(precision_score(y_test, y_pred,average='weighted'),2)*100

In [43]:
TF_IDF_RE=  round(recall_score(y_test, y_pred, average='weighted'),2)*100

In [44]:
TF_IDF_F1 =  round(f1_score(y_test, y_pred, average='weighted'),2)*100

## TF-IDF WITH HYPER PARAMETER 

## VECTORIZATION WITH HYPERPARAMETER TUNING 

In [45]:
# Performing Vectorization using sublinear_tf=True,max_df=0.5,min_df=1,stop_words=stopwords.words('english'),use_idf=True,smooth_idf=True
myvector2=TfidfVectorizer(sublinear_tf=True,
                          max_df=0.95,
                          min_df=1,
                          use_idf=True,smooth_idf=True)
train_vector2=myvector2.fit_transform(X_train)

In [46]:
train_vector2.shape

(1454220, 109944)

In [47]:
# We see that a vocabulary of 109944 words is formed

In [48]:
test_vector2=myvector2.transform(X_test)

In [49]:
test_vector2.shape

(363555, 109944)

## MODELLiNG

In [50]:
# Perform classification using MultinominalNB()
clf=MultinomialNB()
train=clf.fit(train_vector2,y_train)

In [51]:
# Check for predicted value of y
y_pred2=clf.predict(test_vector2)

In [52]:
# Classification Report
print(classification_report(y_test, y_pred2))

                   precision    recall  f1-score   support

__label__Negative       0.86      0.06      0.11     29993
 __label__Neutral       0.31      0.00      0.00     23977
 __label__Postive       0.86      1.00      0.92    309585

         accuracy                           0.86    363555
        macro avg       0.68      0.35      0.35    363555
     weighted avg       0.82      0.86      0.79    363555



In [53]:
# Confusion Matrix
confusion_matrix(y_test,y_pred2)

array([[  1826,      4,  28163],
       [   158,      9,  23810],
       [   141,     16, 309428]], dtype=int64)

In [54]:
# Evaluation Metrics
print('accuracy:', accuracy_score(y_test, y_pred2))
print('recall:', recall_score(y_test, y_pred2, average='weighted'))
print('f1-score:', f1_score(y_test, y_pred2, average='weighted'))
print('precision:', precision_score(y_test, y_pred2,average='weighted'))

accuracy: 0.8561648168777765
recall: 0.8561648168777765
f1-score: 0.7948211096641231
precision: 0.820446959072453


In [55]:
df_compare=pd.DataFrame()

In [56]:
df_compare['Model']=['Model without hyperparameters','Model with hyperparameters']
df_compare['Accuracy']=[accuracy_score(y_test, y_pred),accuracy_score(y_test, y_pred2)]
df_compare['Recall']=[recall_score(y_test, y_pred, average='weighted'),recall_score(y_test, y_pred2, average='weighted')]
df_compare['F1-Score']=[f1_score(y_test, y_pred, average='weighted'),f1_score(y_test, y_pred2, average='weighted')]
df_compare['Precision']=[precision_score(y_test, y_pred,average='weighted'),precision_score(y_test, y_pred2,average='weighted')]

In [57]:
df_compare

,Model,Accuracy,Recall,F1-Score,Precision
0,Model without hyperparameters,0.856093,0.856093,0.794577,0.826628
1,Model with hyperparameters,0.856165,0.856165,0.794821,0.820447


In [58]:
TFIDF_HYPER_ACC =  round(accuracy_score(y_test, y_pred2),2)*100

In [59]:
TFIDF_HYPER_PREC = round(precision_score(y_test, y_pred2,average='weighted'),2)*100

In [60]:
TFIDF_HYPER_RE = round(recall_score(y_test, y_pred2, average='weighted'),2)*100

In [61]:
TFIDF_HYPER_F1 = round(f1_score(y_test, y_pred2, average='weighted'),2)*100

### BAG OF WORDS 

In [62]:
from sklearn.feature_extraction.text import CountVectorizer


In [63]:
# Vectorizing the training data using CountVectorizer()
cv = CountVectorizer()
train_vector_bw=cv.fit_transform(X_train)


train_vector_bw.shape
# We see that a vocabulary of 110244 words are made


test_vector_bw=cv.transform(X_test)

test_vector_bw.shape

(363555, 109944)

In [64]:
# Perform classification using MultinominalNB()
clf=MultinomialNB()
train=clf.fit(train_vector_bw,y_train)

# Check for predicted value of y
y_pred3=clf.predict(test_vector_bw)

print(("**"*50))

# Classification Report
print(classification_report(y_test, y_pred3))

print(("**"*50))


# Confusion Matrix
print(confusion_matrix(y_test,y_pred3))


print(("**"*50))

# Evaluation Metrics
print('accuracy:', accuracy_score(y_test, y_pred3))
print('recall:', recall_score(y_test, y_pred3, average='weighted'))
print('f1-score:', f1_score(y_test, y_pred3, average='weighted'))
print('precision:', precision_score(y_test, y_pred3,average='weighted'))

****************************************************************************************************
                   precision    recall  f1-score   support

__label__Negative       0.57      0.43      0.49     29993
 __label__Neutral       0.27      0.10      0.15     23977
 __label__Postive       0.90      0.96      0.93    309585

         accuracy                           0.86    363555
        macro avg       0.58      0.50      0.52    363555
     weighted avg       0.83      0.86      0.84    363555

****************************************************************************************************
[[ 12885   1994  15114]
 [  3026   2400  18551]
 [  6652   4551 298382]]
****************************************************************************************************
accuracy: 0.862777296420074
recall: 0.862777296420074
f1-score: 0.8420700537145639
precision: 0.8300217335101121


In [65]:
BOW_UNI_ACC =  round(accuracy_score(y_test, y_pred3),2)*100

In [66]:
BOW_UNI_PREC =  round(precision_score(y_test, y_pred3,average='weighted'),2)*100

In [67]:
BOW_UNI_RE =  round(recall_score(y_test, y_pred3, average='weighted'),2)*100

In [68]:
BOW_UNI_F1 =  round( f1_score(y_test, y_pred3, average='weighted'),2)*100

# N-GRAM

In [69]:
# Vectorizing the training data using CountVectorizer()
cv = CountVectorizer(ngram_range=(2,2))
train_vector_bi=cv.fit_transform(X_train)


train_vector_bi.shape
# We see that a vocabulary of 110244 words are made


test_vector_bi=cv.transform(X_test)

test_vector_bi.shape

(363555, 6109630)

In [70]:
# Perform classification using MultinominalNB()
clf=MultinomialNB()
train=clf.fit(train_vector_bi,y_train)

# Check for predicted value of y
y_pred4=clf.predict(test_vector_bi)

print(("**"*50))

# Classification Report
print(classification_report(y_test, y_pred4))

print(("**"*50))


# Confusion Matrix
print(confusion_matrix(y_test,y_pred4))


print(("**"*50))

# Evaluation Metrics
print('accuracy:', accuracy_score(y_test, y_pred4))
print('recall:', recall_score(y_test, y_pred4, average='weighted'))
print('f1-score:', f1_score(y_test, y_pred4, average='weighted'))
print('precision:', precision_score(y_test, y_pred4,average='weighted'))

****************************************************************************************************
                   precision    recall  f1-score   support

__label__Negative       0.81      0.17      0.29     29993
 __label__Neutral       0.61      0.02      0.04     23977
 __label__Postive       0.87      1.00      0.93    309585

         accuracy                           0.87    363555
        macro avg       0.76      0.40      0.42    363555
     weighted avg       0.85      0.87      0.82    363555

****************************************************************************************************
[[  5224    158  24611]
 [   613    496  22868]
 [   626    157 308802]]
****************************************************************************************************
accuracy: 0.8651290726300009
recall: 0.8651290726300009
f1-score: 0.8161119060660014
precision: 0.845088337532923


In [71]:
BOW_BI_ACC = round(accuracy_score(y_test, y_pred4),2)*100

In [72]:
BOW_BI_ACC

87.0

In [73]:
BOW_BI_PREC =  round(precision_score(y_test, y_pred4,average='weighted'),2)*100

In [74]:
BOW_BI_RE =  round(recall_score(y_test, y_pred4, average='weighted'),2)*100

In [75]:
BOW_BI_F1 =  round(f1_score(y_test, y_pred4, average='weighted'),2)*100

In [76]:
comparision = pd.DataFrame()

In [77]:
comparision['MODEL NAME'] = ['MultinomialNB (NAIEVE_BAYES)','MultinomialNB (NAIEVE_BAYES)','MultinomialNB (NAIEVE_BAYES)','MultinomialNB (NAIEVE_BAYES)']

In [78]:
comparision['TEXT EXTRACTION METHOD'] = ['TF_IDF','TF_IDF_HYPERPARAMETRE','BAG OF WORDS(UNIGRAM)','BAG OF WORDS(BIGRAM)']

In [79]:
comparision['ACCURACY'] = [TF_IDF_ACC,TFIDF_HYPER_ACC,BOW_UNI_ACC,BOW_BI_ACC]

In [80]:
comparision['PRECESSION'] = [TF_IDF_PREC,TFIDF_HYPER_PREC,BOW_UNI_PREC,BOW_BI_PREC]

In [81]:
comparision['RECALL'] = [TF_IDF_RE,TFIDF_HYPER_RE,BOW_UNI_RE,BOW_BI_RE]

In [82]:
comparision['F1_SCORE'] = [TF_IDF_F1,TFIDF_HYPER_F1,BOW_UNI_F1,BOW_BI_F1]

In [83]:
comparision

,MODEL NAME,TEXT EXTRACTION METHOD,ACCURACY,PRECESSION,RECALL,F1_SCORE
0,MultinomialNB (NAIEVE_BAYES),TF_IDF,86.0,83.0,86.0,79.0
1,MultinomialNB (NAIEVE_BAYES),TF_IDF_HYPERPARAMETRE,86.0,82.0,86.0,79.0
2,MultinomialNB (NAIEVE_BAYES),BAG OF WORDS(UNIGRAM),86.0,83.0,86.0,84.0
3,MultinomialNB (NAIEVE_BAYES),BAG OF WORDS(BIGRAM),87.0,85.0,87.0,82.0


In [84]:
c1=(86+83+86+79)/4                # average score 
c1

83.5

In [85]:
c2=(86+82+86+79)/4
c2

83.25

In [86]:
c3=(86+83+86+84)/4
c3

84.75

In [87]:
c4=(87+85+87+82)/4
c4

85.25

# *************patio lawn gardern data********************

In [88]:
df_patio=pd.read_csv('PATIO_COMPLETELY_CLEAN_FULLY_MERGED_RDY_FOR_MDLLING')

In [89]:
df_patio.head()

,title,brand,main_cat,price,product_id,Reviewer_id,rating,verified,reviewText,DATE
0,Colorado US State Flag: 3x5foot poly,3x5fFlags,Amazon Home,52.730167,9539723809,AUVPE2KEXBJVT,5.0,True,I don't spend a lot on my flags because they r...,2011-12-11
1,Colorado US State Flag: 3x5foot poly,3x5fFlags,Amazon Home,52.730167,9539723809,A2F6GES1MBOFXS,5.0,True,"Super fast processing and shipping, if you are...",2011-07-30
2,Colorado US State Flag: 3x5foot poly,3x5fFlags,Amazon Home,52.730167,9539723809,A1SDVD3SZI1BAK,5.0,True,Great product. I would recommend this product...,2016-05-04
3,Colorado US State Flag: 3x5foot poly,3x5fFlags,Amazon Home,52.730167,9539723809,A1ZQJ3KCSLUPR3,5.0,True,GREAT PRICE I LOVE MY STATE AND COUNTRY,2015-11-02
4,Colorado US State Flag: 3x5foot poly,3x5fFlags,Amazon Home,52.730167,9539723809,ANU8FBZM618M3,5.0,True,Great display flag for the den.,2015-09-01


In [90]:
df_patio.shape

(932665, 10)

In [91]:
df_patio.drop_duplicates(subset=None,keep='first',inplace=True)    #is is used to drop duplicate column remain first

In [92]:
df_patio.shape

(727480, 10)

In [93]:
sentiment=[]
for i in df_patio['rating']:
    if (i>3):
        sentiment.append('Positive')
    elif(i<3):
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')

In [94]:
df_patio['ReviewSentiment']=sentiment

In [95]:
df_patio.head()

,title,brand,main_cat,price,product_id,Reviewer_id,rating,verified,reviewText,DATE,ReviewSentiment
0,Colorado US State Flag: 3x5foot poly,3x5fFlags,Amazon Home,52.730167,9539723809,AUVPE2KEXBJVT,5.0,True,I don't spend a lot on my flags because they r...,2011-12-11,Positive
1,Colorado US State Flag: 3x5foot poly,3x5fFlags,Amazon Home,52.730167,9539723809,A2F6GES1MBOFXS,5.0,True,"Super fast processing and shipping, if you are...",2011-07-30,Positive
2,Colorado US State Flag: 3x5foot poly,3x5fFlags,Amazon Home,52.730167,9539723809,A1SDVD3SZI1BAK,5.0,True,Great product. I would recommend this product...,2016-05-04,Positive
3,Colorado US State Flag: 3x5foot poly,3x5fFlags,Amazon Home,52.730167,9539723809,A1ZQJ3KCSLUPR3,5.0,True,GREAT PRICE I LOVE MY STATE AND COUNTRY,2015-11-02,Positive
4,Colorado US State Flag: 3x5foot poly,3x5fFlags,Amazon Home,52.730167,9539723809,ANU8FBZM618M3,5.0,True,Great display flag for the den.,2015-09-01,Positive


In [96]:
text_patio  = df_patio[["reviewText","ReviewSentiment"]]

In [97]:
text_patio

,reviewText,ReviewSentiment
0,I don't spend a lot on my flags because they r...,Positive
1,"Super fast processing and shipping, if you are...",Positive
2,Great product. I would recommend this product...,Positive
3,GREAT PRICE I LOVE MY STATE AND COUNTRY,Positive
4,Great display flag for the den.,Positive
...,...,...
932660,perfect for the project at hand,Positive
932661,Like,Positive
932662,Just what I needed to use in my planting. Just...,Positive
932663,Perfect for what I needed it for. My bonsais ...,Positive


In [98]:
text_patio.dtypes

reviewText         object
ReviewSentiment    object
dtype: object

In [99]:
text_patio.isnull().sum()

reviewText         0
ReviewSentiment    0
dtype: int64

# Text Preprocessing

In [100]:
import spacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
stopwords = en.Defaults.stop_words

In [101]:
def clean_text(text):
    # Split the text into individual words
    words = text.split()
    
    # Remove any punctuation or non-alphabetic characters
    words = [word for word in words if word.isalpha()]
    
    # Convert all words to lowercase
    words = [word.lower() for word in words]
    
    # Remove any stop words
    stop_words = stopwords=('english')
    words = [word for word in words if word not in stop_words]
    
    # Join the words back into a single string
    return " ".join(words)


# Clean the "text" column using the clean_text() function
text_patio["reviewText"] = text_patio["reviewText"].apply(clean_text)

# Print the cleaned data
print(text_patio)

                                               reviewText ReviewSentiment
0       spend a lot on my flags because they really ge...        Positive
1       super fast processing and if you are looking f...        Positive
2         great would recommend this would recommend this        Positive
3                   great price love my state and country        Positive
4                              great display flag for the        Positive
...                                                   ...             ...
932660                    perfect for the project at hand        Positive
932661                                               like        Positive
932662       just what needed to use in my just the right        Positive
932663  perfect for what needed it my bonsais humidity...        Positive
932664  very small no more than a cup of stones cheape...        Negative

[727480 rows x 2 columns]


In [102]:
text_patio["reviewText"]=text_patio["reviewText"].apply(lambda x:' '.join([word for word in x.split() if word not in stopwords]))

### Checking the first 100 review texts

In [103]:
c = 0 
for i in text_patio["reviewText"]:
    print(i)
    c+=1
    if c==100:
        break

spend lot flags beat lesser quality figuring replaced couple times year fact inexpensive flag flying months later pleases sees winds averaging mph day occasional storm friont gusts reaching high far months starting little fraying buying shortly ready finally gives
super fast processing looking flags look good quality product great company business
great recommend recommend
great price love state country
great display flag
cheap polyester flag serves purpose school color vibrant size true x
serves purpose holding little concerned fading think good
great quality looks
loved
knee saver taking vynal tile basement
tool worked remove partial old hardwood works nicely driveway ice frozen snow chipper remover flooring task fiberglass handle ability bend actually assisted removal process unbendable metal need additional buy
gave sharp edge strong
remove old linoleum job lot
bit big use worked got old vinyl great help removing carpet pad stuck
heavy heavy good tool hand
good
tool saved purchased

### We see that our text is now clean and ready for modelling

### When you train a fasttext model, it expects labels to be specified with label prefix. We will just create a third column in the dataframe that has label as well as the review description


In [104]:
text_patio['ReviewSentiment'] = '__label__' + text_patio['ReviewSentiment'].astype(str)
text_patio.head(5)

,reviewText,ReviewSentiment
0,spend lot flags beat lesser quality figuring r...,__label__Positive
1,super fast processing looking flags look good ...,__label__Positive
2,great recommend recommend,__label__Positive
3,great price love state country,__label__Positive
4,great display flag,__label__Positive


In [105]:
text_patio.shape

(727480, 2)

In [106]:
X=[]
y=[]
for i in text_patio['reviewText']:
    X.append(i)
for j in text_patio['ReviewSentiment']:
    y.append(j)  

In [107]:
print(type(X))

<class 'list'>


In [108]:
print(X[:10])

['spend lot flags beat lesser quality figuring replaced couple times year fact inexpensive flag flying months later pleases sees winds averaging mph day occasional storm friont gusts reaching high far months starting little fraying buying shortly ready finally gives', 'super fast processing looking flags look good quality product great company business', 'great recommend recommend', 'great price love state country', 'great display flag', 'cheap polyester flag serves purpose school color vibrant size true x', 'serves purpose holding little concerned fading think good', 'great quality looks', 'loved', 'knee saver taking vynal tile basement']


# TRAIN TEST SPLIT

In [109]:
# Splitting the data in 80:20 ratio for train and test respectively
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=9)

In [110]:
len(X_train)

581984

In [111]:
len(X_test)

145496

In [112]:
len(y_train)

581984

In [113]:
len(y_test)

145496

# VECTORIZATION (TFIDF)

In [114]:
# Vectorizing the training data using TfidfVectorizer()
myvector=TfidfVectorizer()
train_vector=myvector.fit_transform(X_train)

In [115]:
train_vector.shape

(581984, 71855)

In [116]:
test_vector=myvector.transform(X_test)         # here we use same vocabulary of x_train 

In [117]:
test_vector.shape  

(145496, 71855)

## CLASSIFICATION USING NAIVE BAYES

In [118]:
# Perform classification using MultinominalNB()
clf=MultinomialNB()
train=clf.fit(train_vector,y_train)                  # x_train,y_train

In [119]:
# Check for predicted value of y
y_pred=clf.predict(test_vector)                 # on x_test data

In [120]:
# Classification Report
print(classification_report(y_test, y_pred))

                   precision    recall  f1-score   support

__label__Negative       0.88      0.08      0.14     14985
 __label__Neutral       0.52      0.00      0.00     10452
__label__Positive       0.83      1.00      0.91    120059

         accuracy                           0.83    145496
        macro avg       0.75      0.36      0.35    145496
     weighted avg       0.82      0.83      0.76    145496



In [121]:
# Confusion Matrix
confusion_matrix(y_test,y_pred)

array([[  1178,      2,  13805],
       [    94,     11,  10347],
       [    63,      8, 119988]], dtype=int64)

In [122]:
# Evaluation Metrics
print('accuracy:', accuracy_score(y_test, y_pred))
print('recall:', recall_score(y_test, y_pred, average='weighted'))
print('f1-score:', f1_score(y_test, y_pred, average='weighted'))
print('precision:', precision_score(y_test, y_pred,average='weighted'))

accuracy: 0.8328545114642327
recall: 0.8328545114642327
f1-score: 0.7645341519783894
precision: 0.8154146654520638


In [123]:
TF_IDF_ACC = round(accuracy_score(y_test, y_pred),2)*100

In [124]:
TF_IDF_PREC =  round(precision_score(y_test, y_pred,average='weighted'),2)*100

In [125]:
TF_IDF_RE=  round(recall_score(y_test, y_pred, average='weighted'),2)*100

In [126]:
TF_IDF_F1 =  round(f1_score(y_test, y_pred, average='weighted'),2)*100

# TF-IDF WITH HYPER PARAMETER
# VECTORIZATION WITH HYPERPARAMETER TUNING

In [127]:
# Performing Vectorization using sublinear_tf=True,max_df=0.5,min_df=1,stop_words=stopwords.words('english'),use_idf=True,smooth_idf=True
myvector2=TfidfVectorizer(sublinear_tf=True,
                          max_df=0.95,
                          min_df=1,
                          use_idf=True,smooth_idf=True)
train_vector2=myvector2.fit_transform(X_train)

In [128]:
train_vector2.shape

(581984, 71855)

In [129]:
test_vector2=myvector2.transform(X_test)

In [130]:
test_vector2.shape

(145496, 71855)

## MODELLING

In [131]:
# Perform classification using MultinominalNB()
clf=MultinomialNB()
train=clf.fit(train_vector2,y_train)

In [132]:
# Check for predicted value of y
y_pred2=clf.predict(test_vector2)

In [133]:
# Classification Report
print(classification_report(y_test, y_pred2))

                   precision    recall  f1-score   support

__label__Negative       0.88      0.08      0.15     14985
 __label__Neutral       0.52      0.00      0.00     10452
__label__Positive       0.83      1.00      0.91    120059

         accuracy                           0.83    145496
        macro avg       0.75      0.36      0.35    145496
     weighted avg       0.82      0.83      0.76    145496



In [134]:
# Confusion Matrix
confusion_matrix(y_test,y_pred2)

array([[  1192,      2,  13791],
       [    93,     11,  10348],
       [    70,      8, 119981]], dtype=int64)

In [135]:
# Evaluation Metrics
print('accuracy:', accuracy_score(y_test, y_pred2))
print('recall:', recall_score(y_test, y_pred2, average='weighted'))
print('f1-score:', f1_score(y_test, y_pred2, average='weighted'))
print('precision:', precision_score(y_test, y_pred2,average='weighted'))

accuracy: 0.8329026227525156
recall: 0.8329026227525156
f1-score: 0.764705453493407
precision: 0.8151926333394318


In [136]:
df_compare=pd.DataFrame()

In [137]:
df_compare['Model']=['Model without hyperparameters','Model with hyperparameters']
df_compare['Accuracy']=[accuracy_score(y_test, y_pred),accuracy_score(y_test, y_pred2)]
df_compare['Recall']=[recall_score(y_test, y_pred, average='weighted'),recall_score(y_test, y_pred2, average='weighted')]
df_compare['F1-Score']=[f1_score(y_test, y_pred, average='weighted'),f1_score(y_test, y_pred2, average='weighted')]
df_compare['Precision']=[precision_score(y_test, y_pred,average='weighted'),precision_score(y_test, y_pred2,average='weighted')]

In [138]:
df_compare

,Model,Accuracy,Recall,F1-Score,Precision
0,Model without hyperparameters,0.832855,0.832855,0.764534,0.815415
1,Model with hyperparameters,0.832903,0.832903,0.764705,0.815193


In [139]:
TFIDF_HYPER_ACC =  round(accuracy_score(y_test, y_pred2),2)*100

In [140]:
TFIDF_HYPER_PREC = round(precision_score(y_test, y_pred2,average='weighted'),2)*100

In [141]:
TFIDF_HYPER_RE = round(recall_score(y_test, y_pred2, average='weighted'),2)*100

In [142]:
TFIDF_HYPER_F1 = round(f1_score(y_test, y_pred2, average='weighted'),2)*100

# BAG OF WORDS

In [143]:
from sklearn.feature_extraction.text import CountVectorizer


In [144]:
# Vectorizing the training data using CountVectorizer()
cv = CountVectorizer()
train_vector_bw=cv.fit_transform(X_train)


train_vector_bw.shape
# We see that a vocabulary of 110244 words are made


test_vector_bw=cv.transform(X_test)

test_vector_bw.shape

(145496, 71855)

In [145]:
# Perform classification using MultinominalNB()
clf=MultinomialNB()
train=clf.fit(train_vector_bw,y_train)

# Check for predicted value of y
y_pred3=clf.predict(test_vector_bw)

print(("**"*50))

# Classification Report
print(classification_report(y_test, y_pred3))

print(("**"*50))


# Confusion Matrix
print(confusion_matrix(y_test,y_pred3))


print(("**"*50))

# Evaluation Metrics
print('accuracy:', accuracy_score(y_test, y_pred3))
print('recall:', recall_score(y_test, y_pred3, average='weighted'))
print('f1-score:', f1_score(y_test, y_pred3, average='weighted'))
print('precision:', precision_score(y_test, y_pred3,average='weighted'))

****************************************************************************************************
                   precision    recall  f1-score   support

__label__Negative       0.59      0.43      0.50     14985
 __label__Neutral       0.29      0.08      0.12     10452
__label__Positive       0.88      0.96      0.92    120059

         accuracy                           0.84    145496
        macro avg       0.58      0.49      0.51    145496
     weighted avg       0.80      0.84      0.82    145496

****************************************************************************************************
[[  6507    618   7860]
 [  1398    810   8244]
 [  3201   1412 115446]]
****************************************************************************************************
accuracy: 0.8437551547808875
recall: 0.8437551547808875
f1-score: 0.8173547643783727
precision: 0.8049872844807807


In [146]:
BOW_UNI_ACC =  round(accuracy_score(y_test, y_pred3),2)*100

In [147]:
BOW_UNI_PREC =  round(precision_score(y_test, y_pred3,average='weighted'),2)*100

In [148]:
BOW_UNI_RE =  round(recall_score(y_test, y_pred3, average='weighted'),2)*100

In [149]:
BOW_UNI_F1 =  round( f1_score(y_test, y_pred3, average='weighted'),2)*100

# N-GRAM

In [150]:
# Vectorizing the training data using CountVectorizer()
cv = CountVectorizer(ngram_range=(2,2))
train_vector_bi=cv.fit_transform(X_train)


train_vector_bi.shape
# We see that a vocabulary of 110244 words are made


test_vector_bi=cv.transform(X_test)

test_vector_bi.shape

(145496, 3246637)

In [151]:
# Perform classification using MultinominalNB()
clf=MultinomialNB()
train=clf.fit(train_vector_bi,y_train)

# Check for predicted value of y
y_pred4=clf.predict(test_vector_bi)

print(("**"*50))

# Classification Report
print(classification_report(y_test, y_pred4))

print(("**"*50))


# Confusion Matrix
print(confusion_matrix(y_test,y_pred4))


print(("**"*50))

# Evaluation Metrics
print('accuracy:', accuracy_score(y_test, y_pred4))
print('recall:', recall_score(y_test, y_pred4, average='weighted'))
print('f1-score:', f1_score(y_test, y_pred4, average='weighted'))
print('precision:', precision_score(y_test, y_pred4,average='weighted'))

****************************************************************************************************
                   precision    recall  f1-score   support

__label__Negative       0.83      0.19      0.31     14985
 __label__Neutral       0.65      0.03      0.05     10452
__label__Positive       0.84      1.00      0.91    120059

         accuracy                           0.84    145496
        macro avg       0.77      0.40      0.42    145496
     weighted avg       0.83      0.84      0.79    145496

****************************************************************************************************
[[  2806     71  12108]
 [   276    279   9897]
 [   310     78 119671]]
****************************************************************************************************
accuracy: 0.8437070434926046
recall: 0.8437070434926046
f1-score: 0.7897084901730853
precision: 0.829033588706945


In [152]:
BOW_BI_ACC = round(accuracy_score(y_test, y_pred4),2)*100                  # round the values

In [153]:
BOW_BI_ACC

84.0

In [154]:
BOW_BI_PREC =  round(precision_score(y_test, y_pred4,average='weighted'),2)*100

In [155]:
BOW_BI_RE =  round(recall_score(y_test, y_pred4, average='weighted'),2)*100

In [156]:
BOW_BI_F1 =  round(f1_score(y_test, y_pred4, average='weighted'),2)*100

In [157]:
comparision2 = pd.DataFrame()          #creating dataframe

In [158]:
# creating column name(title)

comparision2['MODEL NAME'] = ['MultinomialNB (NAIVE_BAYES)','MultinomialNB (NAIVE_BAYES)','MultinomialNB (NAIVE_BAYES)','MultinomialNB (NAIVE_BAYES)']

In [159]:
comparision2['TEXT EXTRACTION METHOD'] = ['TF_IDF','TF_IDF_HYPERPARAMETRE','BAG OF WORDS(UNIGRAM)','BAG OF WORDS(BIGRAM)']

In [160]:
comparision2['ACCURACY'] = [TF_IDF_ACC,TFIDF_HYPER_ACC,BOW_UNI_ACC,BOW_BI_ACC]

In [161]:
comparision2['PRECESSION'] = [TF_IDF_PREC,TFIDF_HYPER_PREC,BOW_UNI_PREC,BOW_BI_PREC]

In [162]:
comparision2['RECALL'] = [TF_IDF_RE,TFIDF_HYPER_RE,BOW_UNI_RE,BOW_BI_RE]

In [163]:
comparision2['F1_SCORE'] = [TF_IDF_F1,TFIDF_HYPER_F1,BOW_UNI_F1,BOW_BI_F1]

In [164]:
comparision2

,MODEL NAME,TEXT EXTRACTION METHOD,ACCURACY,PRECESSION,RECALL,F1_SCORE
0,MultinomialNB (NAIVE_BAYES),TF_IDF,83.0,82.0,83.0,76.0
1,MultinomialNB (NAIVE_BAYES),TF_IDF_HYPERPARAMETRE,83.0,82.0,83.0,76.0
2,MultinomialNB (NAIVE_BAYES),BAG OF WORDS(UNIGRAM),84.0,80.0,84.0,82.0
3,MultinomialNB (NAIVE_BAYES),BAG OF WORDS(BIGRAM),84.0,83.0,84.0,79.0


In [165]:
# conclusion:- BOW using ngrams (bigram )has highest precision score than the other model according to that we can consider that model